# Linear Programming with Python (GLOP, SCIP, Gurobi, C-Plex, GLPK, CBC, PuLP)

We will be solving the following set of equations:

$max(x+y)$, subject to:

* $-x+2y\le8$
* $2x+y\le14$
* $2x-y\le10$
* $0\le x\le10$
* $0\le y\le 10$


#### Install Required Packages

In [1]:
# ! pip install -U ortools

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()


✨🍰✨ Everything looks OK!


In [3]:
!conda install pyscipopt

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [4]:
from ortools.linear_solver import pywraplp

## GLOP - Google Linear Optimization Package

#### Create the Solver instance

In [5]:
solver = pywraplp.Solver.CreateSolver('GLOP') # Google Linear Optimization Package

#### Enter the equations

In [6]:
x = solver.NumVar(0, 10, 'x')
y = solver.NumVar(0, 10, 'y')

solver.Add(-x + 2 * y <= 8.0)
solver.Add(2 * x + y <= 14.0)
solver.Add(2 * x - y <= 10.0)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x79c3181bad60> >

#### Enter the objective function

In [7]:
solver.Maximize(x + y)
result = solver.Solve()

#### Get solution

In [8]:
if result == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())

Solution:
Objective value = 10.0
x = 4.0
y = 6.0


## SCIP (Solving Constraint Integer Programs)

In [9]:
from pyscipopt import Model

model = Model("example")

In [10]:
x = model.addVar(vtype="C", name="x")
y = model.addVar(vtype="C", name="y")

model.setObjective(x + y, sense="maximize")

In [11]:
model.addCons(-x + 2 * y <= 8)
model.addCons(2 * x + y <= 14)
model.addCons(2 * x - y <= 10)

c3

In [12]:
model.optimize()

solution = model.getBestSol()
print("Optimal Solution:")
print("x =", solution[x])
print("y =", solution[y])

Optimal Solution:
x = 4.0
y = 6.0


## Gurobi

In [13]:
!pip install gurobipy

In [14]:
! pip install pyomo

In [15]:
! pip install cplex

In [16]:
!conda install -c conda-forge glpk

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [17]:
import pyomo.environ as pyo
import gurobipy as gp
from pyomo.environ import *

In [18]:
model = pyo.ConcreteModel()

model.x = pyo.Var(bounds=(0, 10))
model.y = pyo.Var(bounds=(0, 10))

x = model.x
y = model.y

model.C1 = pyo.Constraint(expr=-x + 2 * y <= 8)
model.C2 = pyo.Constraint(expr=2 * x + y <= 14)
model.C3 = pyo.Constraint(expr=2 * x - y <= 10)

model.obj = pyo.Objective(expr=x + y, sense=maximize)

In [19]:
opt = pyo.SolverFactory('gurobi')
opt.solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': 10.0, 'Upper bound': 10.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 2, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': 0, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': 0.0010340213775634766, 'Error rc': 0}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [20]:
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   4.0 :    10 : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   6.0 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x + y

3 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : - x + 2*y :   8.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y :  14.0 :   True
    C3 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x - y :  10.0 :   True

6 Declarations: x y C1 C2 C3 obj


In [21]:
x = pyo.value(model.x)
y = pyo.value(model.y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = 4.0
y = 6.0


In [22]:
opt = pyo.SolverFactory('glpk')
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 10.0, 'Upper bound': 10.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.004081010818481445}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [23]:
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   4.0 :    10 : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   6.0 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x + y

3 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : - x + 2*y :   8.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y :  14.0 :   True
    C3 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x - y :  10.0 :   True

6 Declarations: x y C1 C2 C3 obj


In [24]:
x = pyo.value(model.x)
y = pyo.value(model.y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = 4.0
y = 6.0


## PuLP - Python Linear Programming

In [25]:
! pip install cython

In [26]:
! pip install pulp

In [27]:
import pulp as pl

In [28]:
model = pl.LpProblem("Simple_LP", pl.LpMaximize)

x = pl.LpVariable("x", lowBound=0,upBound=10)
y = pl.LpVariable("y", lowBound=0,upBound=10)

model += -x + 2 * y <= 8
model += 2 * x + y <= 14
model += 2 * x - y <= 10

model += x + y


In [29]:
status = model.solve()
print(pl.LpStatus[status])

Optimal


In [30]:
x = pl.value(x)
y = pl.value(y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = 4.0
y = 6.0


## An LP Optimization exercise:

Minimize $-4x-2y$ subject to:

* $x+y\le8$
* $8x+3y\ge-24$
* $-6x+8y\le48$
* $3x+5y\le15$
* $x\le3$
* $y\ge0$

In [31]:
model2 = pyo.ConcreteModel()

model2.x = pyo.Var(bounds=(None, 3))
model2.y = pyo.Var(bounds=(0, None))

x = model2.x
y = model2.y

model2.C1 = pyo.Constraint(expr=x + y <= 8)
model2.C2 = pyo.Constraint(expr=8 * x + 3 * y >= -24)
model2.C3 = pyo.Constraint(expr=-6 * x + 8 * y <= 48)
model2.C4 = pyo.Constraint(expr=3 * x + 5 * y <= 15)

model.obj = pyo.Objective(expr=x + y, sense=maximize)

In [32]:
opt = pyo.SolverFactory('gurobi')
opt.solve(model2)

{'Problem': [{'Name': 'x1', 'Lower bound': 1.0, 'Upper bound': 1.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 3, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 3, 'Number of nonzeros': 8, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': 0, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': 0.0005218982696533203, 'Error rc': 0}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [33]:
model2.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  -3.0 :     3 : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False :  Reals

4 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :   8.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None : -24.0 : 8*x + 3*y :  +Inf :   True
    C3 : Size=1, Index=None, Active=True
        Key  : Lower : Body       : Upper : Active
        None :  -Inf : -6*x + 8*y :  48.0 :   True
    C4 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : 3*x + 5*y :  15.0 :   True

6 Declarations: x y C1 C2 C3 C4


In [34]:
x = pyo.value(model2.x)
y = pyo.value(model2.y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = -3.0
y = 0.0


In [35]:
! pyomo help --solvers


Pyomo Solvers and Solver Managers
---------------------------------
Pyomo uses 'solver managers' to execute 'solvers' that perform
optimization and other forms of model analysis.  A solver directly
executes an optimizer, typically using an executable found on the
user's PATH environment.  Solver managers support a flexible mechanism
for asynchronously executing solvers either locally or remotely.  The
following solver managers are available in Pyomo:

    neos       Asynchronously execute solvers on the NEOS server
    serial     Synchronously execute solvers locally

If no solver manager is specified, Pyomo uses the serial solver
manager to execute solvers locally.  The neos solver manager is used
to execute solvers on the NEOS optimization server.


Serial Solver Interfaces
------------------------
The serial manager supports the following solver interfaces:

   +appsi_cbc            2.10.7  Automated persistent interface to Cbc
   +appsi_cplex          22.1.2  Automated persistent 

### CBC Solver

In [36]:
!apt-get install -y coinor-cbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [41]:
model3 = pyo.ConcreteModel()

model3.x = pyo.Var(bounds=(0, 10))
model3.y = pyo.Var(bounds=(0, 10))

x = model3.x
y = model3.y

model3.C1 = pyo.Constraint(expr=-x + 2 * y <= 8)
model3.C2 = pyo.Constraint(expr=2 * x + y <= 14)
model3.C3 = pyo.Constraint(expr=2 * x - y <= 10)

model3.obj = pyo.Objective(expr=x + y, sense=maximize)

In [42]:
opt = pyo.SolverFactory('cbc')
opt.solve(model3)

{'Problem': [{'Name': 'unknown', 'Lower bound': 10.0, 'Upper bound': 10.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of nonzeros': 2, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 2}}, 'Error rc': 0, 'Time': 0.017720937728881836}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [44]:
x = pyo.value(model3.x)
y = pyo.value(model3.y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = 4.0
y = 6.0
